In [40]:
#!/usr/bin/env python
"""
generate_messages_report_by_row_csv.py

This script connects to the database, fetches all messages for conversations
that have at least 2 messages, and generates a CSV report where each message
occupies a separate row. The columns match exactly:

  - Conversation_Start_Date
  - Conversation_ID
  - Message_ID
  - Message_Timestamp
  - Role
  - Content

The CSV is sorted so that the newest conversation (by start date) is on top.

All text fields are enclosed in double quotes.
"""

import csv
import pandas as pd
from sqlalchemy import select, func
from components.db import Session, messages, conversations  # adjust import if necessary

def fetch_valid_conversation_ids(min_messages=2):
    """
    Fetch a list of conversation IDs from the messages table that have at least min_messages.

    Args:
        min_messages (int): The minimum number of messages a conversation must have.

    Returns:
        list: A list of conversation IDs (integers).
    """
    session = Session()
    try:
        stmt = (
            select(
                messages.c.conversation_id,
                func.count(messages.c.message_id).label("message_count")
            )
            .group_by(messages.c.conversation_id)
            .having(func.count(messages.c.message_id) >= min_messages)
        )
        result = session.execute(stmt)
        conversation_ids = [row._mapping["conversation_id"] for row in result]
        return conversation_ids
    except Exception as e:
        print(f"Error fetching conversation IDs: {e}")
        return []
    finally:
        session.close()

def fetch_conversation_start_date(conversation_id):
    """
    Fetch the conversation's start date from the conversations table.

    Args:
        conversation_id (int): The conversation ID.

    Returns:
        The start_time value (datetime) or None if not found.
    """
    session = Session()
    try:
        stmt = select(conversations.c.start_time).where(conversations.c.conversation_id == conversation_id)
        result = session.execute(stmt).first()
        if result:
            return result._mapping["start_time"]
        return None
    except Exception as e:
        print(f"Error fetching start date for conversation {conversation_id}: {e}")
        return None
    finally:
        session.close()

def fetch_messages_for_conversation(conversation_id):
    """
    Query the database for all messages belonging to a specific conversation,
    sorted in ascending order (chronological order).

    Args:
        conversation_id (int): The ID of the conversation.

    Returns:
        list of dict: A list of message dictionaries.
    """
    session = Session()
    try:
        stmt = (
            select(messages)
            .where(messages.c.conversation_id == conversation_id)
            .order_by(messages.c.timestamp.asc())
        )
        result = session.execute(stmt)
        messages_list = [dict(row._mapping) for row in result]
        return messages_list
    except Exception as e:
        print(f"Error fetching messages for conversation {conversation_id}: {e}")
        return []
    finally:
        session.close()

def generate_messages_report_by_row_csv(report_filename="messages_by_row_report.csv"):
    """
    Generate a CSV report where each row represents a single message, with columns:
      - Conversation_Start_Date
      - Conversation_ID
      - Message_ID
      - Message_Timestamp
      - Role
      - Content

    For each conversation with at least 2 messages, the conversation start date,
    conversation ID, and message details are included. The final CSV is sorted
    with the newest conversation on top (descending by conversation start date).

    Args:
        report_filename (str): The filename for the generated CSV report.
    """
    report_rows = []

    # 1) Get all conversation IDs that have at least 2 messages.
    conversation_ids = fetch_valid_conversation_ids(min_messages=2)
    if not conversation_ids:
        print("No valid conversations found. No report generated.")
        return

    # 2) For each conversation, fetch start_date and messages.
    for conv_id in conversation_ids:
        start_date = fetch_conversation_start_date(conv_id)
        if not start_date:
            start_date = "Unknown"

        messages_list = fetch_messages_for_conversation(conv_id)
        for msg in messages_list:
            timestamp = msg.get("timestamp")
            if timestamp:
                # Convert to a readable string:
                timestamp_str = pd.to_datetime(timestamp).strftime("%Y-%m-%d %H:%M:%S")
            else:
                timestamp_str = "Unknown"

            row = {
                "Conversation_Start_Date": str(start_date),
                "Conversation_ID": conv_id,
                "Message_ID": msg.get("message_id", ""),
                "Message_Timestamp": timestamp_str,
                "Role": msg.get("role", ""),
                "Content": msg.get("content", "")
            }
            report_rows.append(row)

    # 3) Create a DataFrame for easy sorting.
    df = pd.DataFrame(report_rows)

    # Sort by start date descending (newest conversation first).
    df.sort_values(by="Conversation_Start_Date", ascending=False, inplace=True)

    # 4) Write to CSV with commas and double quotes around every field.
    try:
        with open(report_filename, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f, delimiter=",", quoting=csv.QUOTE_ALL)
            # Write header row exactly matching your desired columns
            writer.writerow([
                "Conversation_Start_Date",
                "Conversation_ID",
                "Message_ID",
                "Message_Timestamp",
                "Role",
                "Content"
            ])

            # Write each data row
            for _, row in df.iterrows():
                writer.writerow([
                    row["Conversation_Start_Date"],
                    row["Conversation_ID"],
                    row["Message_ID"],
                    row["Message_Timestamp"],
                    row["Role"],
                    row["Content"]
                ])
        print(f"CSV report generated successfully: {report_filename}")
    except Exception as e:
        print(f"Error writing CSV report: {e}")

def main():
    print("Generating messages report in CSV format (only conversations with at least 2 messages)...")
    generate_messages_report_by_row_csv()

if __name__ == "__main__":
    main()


Generating messages report in CSV format (only conversations with at least 2 messages)...
CSV report generated successfully: messages_by_row_report.csv


In [79]:
#!/usr/bin/env python
"""
generate_html_chat.py

Reads a CSV file with columns:
  Conversation_Start_Date, Conversation_ID, Message_ID, Message_Timestamp, Role, Content

Generates an HTML file for viewing the conversations in a collapsible format without JavaScript.
"""

import os
import csv

# ----- Configuration -----
input_filename = "messages_latest_german.csv"  # Make sure this CSV file exists
output_filename = "conversations_jan29_feb05_german.html"

# ----- Step 1: Read the CSV data -----
if not os.path.exists(input_filename):
    print(f"Error: {input_filename} not found!")
    exit(1)

conversations_by_id = {}

with open(input_filename, "r", encoding="utf-8") as csvfile:
    # Create a DictReader; all columns as strings
    reader = csv.DictReader(csvfile)
    
    # Debug: Show detected headers
    print("Detected CSV header keys:", reader.fieldnames)
    
    for row in reader:
        # Extract fields using the exact column names
        start_date = (row.get("Conversation_Start_Date") or "").strip()
        conv_id = (row.get("Conversation_ID") or "").strip()
        msg_id_str = (row.get("Message_ID") or "").strip()
        message_timestamp = (row.get("Message_Timestamp") or "").strip()
        role = (row.get("Role") or "").strip()
        content = (row.get("Content") or "").strip()

        # If any field is missing, warn and skip this row.
        if not (start_date and conv_id and msg_id_str and message_timestamp and role and content):
            print(f"Warning: Skipping row due to missing fields: {row}")
            continue

        try:
            msg_id = int(msg_id_str)
        except ValueError as e:
            print(f"Error parsing Message_ID '{msg_id_str}' in row: {row}\nError: {e}")
            continue

        message = {
            "msg_id": msg_id,
            "timestamp": message_timestamp,
            "role": role,
            "content": content
        }

        if conv_id not in conversations_by_id:
            conversations_by_id[conv_id] = {"start_date": start_date, "messages": []}
        conversations_by_id[conv_id]["messages"].append(message)

# ----- Step 2: Sort messages in each conversation by Message_ID -----
for conv_data in conversations_by_id.values():
    conv_data["messages"].sort(key=lambda msg: msg["msg_id"])

# ----- Step 3: Generate the HTML output (NO JavaScript) -----
html_output = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Complete Solar Conversations Chat (Sorted by Message_ID)</title>
  <style>
    body { margin: 20px; font-family: Arial, sans-serif; }
    details { margin-bottom: 10px; border: 1px solid #ccc; border-radius: 5px; padding: 10px; }
    summary { font-size: 1.1em; font-weight: bold; cursor: pointer; }
    .chat-container { margin-top: 10px; }
    .chat-message { padding: 10px; border-radius: 10px; margin-bottom: 10px; }
    .chat-message.user { background-color: #d1e7dd; text-align: left; }
    .chat-message.assistant { background-color: #cff4fc; text-align: left; }
    .chat-timestamp { font-size: 0.8em; color: #666; margin-top: 5px; }
  </style>
</head>
<body>
  <h1>Solar Conversations ETH Chatbot 29.01 - 05.02</h1>
  <p>Click on a conversation header to reveal the full chat log between the User and Assistant.</p>
"""

# Create a collapsible `<details>` element for each conversation
for conv_id, conv_data in conversations_by_id.items():
    start_date = conv_data["start_date"]
    messages = conv_data["messages"]

    html_output += f"""
    <details>
      <summary>Date: {start_date} | Conversation ID: {conv_id}</summary>
      <div class="chat-container">
    """
    # Add each message as a chat bubble
    for msg in messages:
        role_class = "user" if msg["role"].lower() == "user" else "assistant"
        html_output += f"""
            <div class="chat-message {role_class}">
              <div>{msg['content']}</div>
              <div class="chat-timestamp">Message ID: {msg['msg_id']} | {msg['timestamp']} – {msg['role']}</div>
            </div>
        """
    html_output += """
      </div> <!-- End chat-container -->
    </details>
    """

html_output += """
</body>
</html>
"""

# ----- Step 4: Write the HTML output to a file -----
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f"HTML file '{output_filename}' has been created. Open it in your browser to view the conversations.")


Detected CSV header keys: ['Conversation_Start_Date', 'Conversation_ID', 'Message_ID', 'Message_Timestamp', 'Role', 'Content']
HTML file 'conversations_jan29_feb05_german.html' has been created. Open it in your browser to view the conversations.
